In [ ]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# load MNIST data set
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

n_filters = 32
learn_rate = 0.01
batch_size = 64
epochs = 30000

# define computational graph
with tf.device('/device:CPU:0'):
    # placeholders for images and labels
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])

    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                              strides=[1, 2, 2, 1], padding='SAME')

    # first conv layer
    W_conv1 = weight_variable([3, 3, 1, n_filters])
    b_conv1 = bias_variable([n_filters])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    # second conv layer
    W_conv2 = weight_variable([3, 3, n_filters, n_filters])
    b_conv2 = bias_variable([n_filters])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    # fully connected layer
    W_fc1 = weight_variable([7 * 7 * n_filters, 128])
    b_fc1 = bias_variable([128])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*n_filters])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    # softmax output
    W_fc2 = weight_variable([128, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    error = 1.0 - accuracy

with tf.Session() as sess:
    train_duration = 0
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        # get next random batch for stochastic gradient descent
        batch = mnist.train.next_batch(batch_size)
        if i % 100 == 0:
            # compute error over entire train set, but batchwise
            #train_error = 0
            #j = 0
            #n_trb = mnist.train.images.shape[0] // 100
            #while j < n_trb:
                #trb_x = mnist.train.images[j*100:(j+1)*100]
                #trb_y = mnist.train.labels[j*100:(j+1)*100]
                #train_error += error.eval(feed_dict={x: trb_x, y_: trb_y})
                #j += 1
            #train_error /= n_trb
            # compute error over entire validation set, but batchwise
            valid_error = 0
            j = 0
            n_vb = mnist.validation.images.shape[0] // 100
            while j < n_vb:
                vb_x = mnist.validation.images[j*100:(j+1)*100]
                vb_y = mnist.validation.labels[j*100:(j+1)*100]
                valid_error += error.eval(feed_dict={x: vb_x, y_: vb_y})
                j += 1
            valid_error /= n_vb 
            print("{} {:.5f}".format(i, valid_error))
        t_start = time.time()
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        train_duration += time.time() - t_start

    # compute error over entire test set, but batchwise
    test_error = 0
    j = 0
    n_teb = mnist.test.images.shape[0] // 100
    while j < n_teb:
        teb_x = mnist.test.images[j*100:(j+1)*100]
        teb_y = mnist.test.labels[j*100:(j+1)*100]
        test_error += error.eval(feed_dict={x: teb_x, y_: teb_y})
        j += 1
    test_error /= n_teb
    print("test error {:.5f}".format(test_error))
    print("train_duration: {:.1f}s".format(train_duration))
        